# 🟡Step 0: Imports


**Most Relevant Papers** <br />
https://arxiv.org/pdf/1407.5675.pdf <br />
https://arxiv.org/pdf/1701.08784.pdf

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


%matplotlib inline
from IPython.display import display
import time

# Possibly Redundant
from scipy import ndimage, misc
from skimage.feature import peak_local_max
from skimage import data, img_as_float

# 🟡Step 1: Read the data (tar.gz file)
As a first step, we unzipped the tar.gz file into a .dat file using 7-zip. 
Then, we convert the .dat file into a string and then into a DataFrame.

.strip() --> remove spaces on the sides

.split() --> separate values by spaces (otherwise we'd get a single conlumn)

In [2]:
# Convert .dat file into string (list comprehension)
datContent = [i.strip().split() for i in open("tth_semihad.dat").readlines()]

# Convert list into DataFrame
mydata = pd.DataFrame(datContent)

# 🟡Step 2: Explore the data
**Physics**

Jonas: "The file was produced from a simulation of pp->tt~H where the top decays hadronically
and the anti-top decays leptonically. <br /> I selected events with exactly 1 fat jet with R=1.5."


**Notes**
- The rows represent events (of 1 fat jet each, R = 1.5) 
- The first column represents the number of constituents of the jet  
- The following columns represent the coordinates of the constituents, η, φ, pT, cycling in that order. <br />(e.g. columns 1, 2, 3 are η, φ, pT for the 1st constituent, columns 4, 5, 6 are η, φ, pT for the 2nd constituent etc.)

<br />

- -infinity < η < infinity 
- -π < φ < π
- pT[GeV] > 0



In [3]:
# Display the data
mydata = mydata.rename(columns={0: 'Const'})
display(mydata.head())

# Print statements
events = mydata.shape[0]
print('There are {} rows (events).'.format(events))
print('The maximum number of constituents in an event is {}.'.format((mydata.shape[1] - 1) // 3))

# Display data types
#print('\nData Types: \n', mydata.dtypes)

# Descriptive statistics on data
#mydata.describe()

,Const,1,2,3,4,5,6,7,8,9,...,99,100,101,102,103,104,105,106,107,108
0,4,2.30474,0.221042,78.9436,1.00519,0.736657,61.9115,1.25546,0.748395,48.9755,...,None,None,None,None,None,None,None,None,None,None
1,2,2.35134,-2.18449,176.076,2.46233,-1.50073,47.3355,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,6,0.492933,0.766876,51.5247,-0.984489,2.29985,13.7463,0.103217,1.40088,5.31666,...,None,None,None,None,None,None,None,None,None,None
3,10,-0.624329,0.566723,130.197,-0.602316,0.573666,38.5226,-0.541426,0.449072,15.3244,...,None,None,None,None,None,None,None,None,None,None
4,15,-0.538961,-0.617644,0.819517,0.527734,1.53319,1.94989,0.20174,0.916744,5.63418,...,None,None,None,None,None,None,None,None,None,None


There are 12177 rows (events).
The maximum number of constituents in an event is 36.


# 🟡Step 3: Construct Average Jet Image

**Jonas:**

The data in each row represents all the constituents of a fat jet in the original (φ,η,pT) coordinate system of the detector. 
You should now transform all the constituents of one fat jet (i.e. individually for each row in the data file) into a new coordinate system 
(φ',η’,pT). Consequently you transform/bin this information into a two-dimensional image/Heat Map.   

The steps in detail are

1. Find the constituent with the highest pT
2. Choose the center of the new coordinate system such that this constituent has the coordinates (φ’,η’)=0. This operation corresponds to rotating and boosting along the beam direction to center the jet.  
3. Rotate all constituents around (φ’,η’)=0 such that the constituent with the 2nd highest pT is at 12 o’clock, i.e. at  (φ’,η’)=(0,e) with e > 0.
4. Flip all the constituents such that the constituents with the 3rd highest pT is on the right, i.e. at (φ’,η’)=(f,e) with f > 0
5. Define a 2-dim image with 40x40 pixels which correspond to η', φ' ∈ (−R, R). The jets we are looking at are approximately cone-shaped with radius R. I.e. all the constituents of the jet should end up within this image. For each pixel you now store the sum of the pT of all constituents that are located within this pixel. I.e. you discretise the (φ’,η’) coordinates into pixels and the intensity of the pixel corresponds to the sum of the energy of all constituents in the pixel.

As a very first step you can plot the images constructed from just steps 1)+2)+5) for a couple of jets. All the images will only have a few pixels filled. Subsequently you should plot the sum of all these images. This should give a broad peak at the center. 

Finally you should apply 1)-5) and plot the sum. This should hopefully look like the image from the reference. 



---
<br />

---
<br />

---
<br />

---
<br />

---

### 🔵 0. Define image plotting functions

#### 🔴 Helper Function that creates an image of EVENT

(bins η φ p first)

In [4]:
def plot_event(event1, R=1.5, pixels=60):
    
    '''
    Displays an image of a pandas Series (event)
    
    Input: event (row)
    Output: null, just plots image
    '''
    
    # Create copy of event so that it's not accidentally modified
    event = event1.copy(deep=True)
    
    # Initiate bin lists
    bin_h = []
    bin_f = []
    bin_p = []
    
    # Define max number of constituents 
    max_const = event.shape[0] // 3
    # For all constituents
    for i in range(max_const):
        # Add constituent's η, φ, p to bins
        bin_h.append(list(event.iloc[::3])[i])
        bin_f.append(list(event.iloc[1::3])[i])
        bin_p.append(list(event.iloc[2::3])[i])

    # Turn lists into Series
    bin_h = pd.Series(bin_h)
    bin_f = pd.Series(bin_f)
    bin_p = pd.Series(bin_p)
    
    # Define no. of bins
    bin_count = np.linspace(-R, R, pixels + 1)
    
    # Create bins from -R to R and convert to DataFrame
    bins = np.histogram2d(bin_f, bin_h, bins=bin_count, weights=bin_p)[0]
    bins = pd.DataFrame(bins)
    
    # Plot Image
    sns.heatmap(bins)
    plt.title('(φ\', η\') ∈ [-R, R] with R = {}'.format(R))
    plt.xlabel('φ\' pixels')
    plt.ylabel('η\' pixels')
    plt.show()
    

#### 🔴 Helper Function that creates an image of DATAFRAME

In [5]:
def plot_dataframe(df1, R=1.5, pixels=60):
    
    '''
    Displays an image of a pandas DataFrame (many events, average image)
    
    Input: dataframe (multiple events)
    Output: null, just plots image
    '''
    
    # Create copy of df so that it's not accidentally modified
    df = df1.copy(deep=True)

    # Initiate bin lists
    bin_h = []
    bin_f = []
    bin_p = []

    # Define max number of constituents 
    max_const = df.shape[1] // 3

    # For all rows
    for i in range(df.shape[0]):             

        # For all constituents
        for j in range(max_const):
            # Add constituent's η to η bin
            bin_h.append(list(df.loc[i][::3])[j])
            bin_f.append(list(df.loc[i][1::3])[j])
            bin_p.append(list(df.loc[i][2::3])[j])

    # Turn lists into Series
    bin_h = pd.Series(bin_h)
    bin_f = pd.Series(bin_f)
    bin_p = pd.Series(bin_p)

   # Define no. of bins
    bin_count = np.linspace(-R, R, pixels + 1)

    # Create bins from -R to R (using bins vector)
    bins = np.histogram2d(bin_f, bin_h, bins=bin_count, weights=bin_p)[0]

    # Convert to DataFrame
    bins = pd.DataFrame(bins)

    # Plot Heat Map
    sns.heatmap(bins)
    plt.title('(φ\', η\') ∈ [-R, R] with R = {}'.format(R))
    plt.xlabel('φ\' pixels')
    plt.ylabel('η\' pixels')
    plt.show()

---
<br />

---
<br />

---
<br />

---
<br />

---

### 🔵 1. Extract Maxima
For each row, extract the maximum pT and its corresponding η and φ. <br />
We also extract the 2nd and 3rd maximum pT's for future use. 

---

#### 🔴 Mild Pre-Processing

1. Define function to separate constituents from coordinates, deal with NaN values, and convert values to floats.
2. Define processed dataframe and constituents vector

In [6]:
def prep(df):
    '''
    Input: DataFrame to be transformed
    Output: Transformed DataFrame, constituents Series 
    '''
    
    # Create df copy
    df1 = df.copy(deep=True)
    
    # Extract constituents column
    const = df1['Const']
    # Drop constituents from df
    df1 = df1.drop('Const', axis=1)
    
    # Replace NaN with 0
    df1 = df1.fillna(0)

    # Convert values to floats
    df1 = df1.astype(float)
    
    return df1, const

In [7]:
# Define processed dataframe and constituents vector
mydata_prep, const = prep(mydata)

---

#### 🔴 Define Helper Function that returns 3 vectors, one for each pT and its η, φ. (For the three maximum pT's)

- **1st vector**: 1st maximum pT and its η, φ
- **2nd vector**: 2nd maximum pT and its η, φ
- **3rd vector**: 3rd maximum pT and its η, φ

In [8]:
def extract_max123(event1):
    
    '''
    Input: event (row). 
    e.g. mydata_prep.iloc[0]
    
    Output[0]: [Series of 1st max p, φ, η]
    Output[1]: [Series of 2nd max p, φ, η]
    Output[2]: [Series of 3rd max p, φ, η]
    '''
    
    
    # Create event copy
    event = event1.copy(deep=True)
    
    # Separate η, φ, pT
    hdata = event[::3]
    fdata = event[1::3]
    pdata1 = event[2::3]
    
    
    
    
    # 1. Extract index of maximum pT
    maxid1 = pdata1.idxmax()
    maxlist1 = []

    # 2. Extract max η, φ, pT for event
    if pdata1.max() != 0:                                                                     # Brief explanation of if statement below)
        maxlist1.append([event.iloc[maxid1-1], event.iloc[maxid1-2], event.iloc[maxid1-3]])   # From event, add to list the max pT and its η, φ
    else:
        maxlist1.append([0., event.iloc[maxid1-2], event.iloc[maxid1-3]])                    # If max pT is 0, then add it as 0 and not the first value
            
    # 3. Create & Display dataframe of max pT, η, φ
    row_max1 = pd.Series(data=maxlist1[0], index=['pT', 'φ', 'η'])
    
    
    
    
    
    # 0. Set Max pT to 0 to find next Max pT
    pdata2 = pdata1.copy(deep=True)
    pdata2.loc[maxid1] = 0

    # 1. Extract index of maximum pT
    maxid2 = pdata2.idxmax()
    maxlist2 = []

    # 2. Extract max η, φ, pT for event
    if pdata2.max() != 0:                                                                     # Brief explanation of if statement below)
        maxlist2.append([event.iloc[maxid2-1], event.iloc[maxid2-2], event.iloc[maxid2-3]])   # From event, add to list the max pT and its η, φ
    else:
        maxlist2.append([0., event.iloc[maxid2-2], event.iloc[maxid2-3]])                    # If max pT is 0, then add it as 0 and not the first value
            
    # 3. Create & Display dataframe of max pT, η, φ
    row_max2 = pd.Series(data=maxlist2[0], index=['pT', 'φ', 'η'])
    
    
    
    
    
    # 0. Set Max pT to 0 to find next Max pT
    pdata3 = pdata2.copy(deep=True)
    pdata3.loc[maxid2] = 0

    # 1. Extract index of maximum pT
    maxid3 = pdata3.idxmax()
    maxlist3 = []

    # 2. Extract max η, φ, pT for event
    if pdata3.max() != 0:                                                                     # Brief explanation of if statement below)
        maxlist3.append([event.iloc[maxid3-1], event.iloc[maxid3-2], event.iloc[maxid3-3]])   # From event, add to list the max pT and its η, φ
    else:
        maxlist3.append([0., event.iloc[maxid3-2], event.iloc[maxid3-3]])                    # If max pT is 0, then add it as 0 and not the first value
            
    # 3. Create & Display dataframe of max pT, η, φ
    row_max3 = pd.Series(data=maxlist3[0], index=['pT', 'φ', 'η'])
    
    
    
    return row_max1, row_max2, row_max3

**Why the if statement?** <br />
Because if maximum pT is 0 in the pdata vector, it picks the ID of the first pT by default as the max (because they're all 0). <br />
Then, it goes to the non-zero'd event vector and adds its non-zero pT as the max, when the value of that max should clearly have been 0.

So the if statement fixes this: <br />
- If max pT != 0, then add it as normal.
- If max pT = 0, then add '0' as its value instead. (with the coordinates of the first pT, which is incorrect, but this doesn't matter since pT = 0 are not taken into account in the image) <br />


---

#### 🔴 Define 3 DataFrames containing the max p, η, φ for all events 

Try this using the map() function instead

In [9]:
maxpt1 = []
maxpt2 = []
maxpt3 = []

start = time.time()

# For all events, add maxima to & coordinates to list
for i in range(events):
    maxpt1.append(extract_max123(mydata_prep.iloc[i])[0])
    maxpt2.append(extract_max123(mydata_prep.iloc[i])[1])
    maxpt3.append(extract_max123(mydata_prep.iloc[i])[2])
    
# Turn lists into DataFrames
max1 = pd.DataFrame(maxpt1)
max2 = pd.DataFrame(maxpt2)
max3 = pd.DataFrame(maxpt3)

# Create list of DataFrames containing all df for the 3 maxima
max123 = [max1, max2, max3]

end = time.time()
print('Time taken: {0:.2f}s'.format(end-start))

Time taken: 44.34s


---
<br />

---
<br />

---
<br />

---
<br />

---

### 🔵2. Shift: Centre New Coordinate System
For each row, we centre a new coordinate system so that the highest pT constituent's coordinates are (φ', η') = (0, 0). <br />
This corresponds to rotating and boosting along the beam direction to center the jet.

How does η transform? We need a Lorentz Transformation. 

**Paper** (E) <br />
Histograms binned in
either the angular separation of events or the rapidity separation of events can
be contributed to by events whose centre of mass frames are boosted by arbitrary velocities with respect to the rest frame of the detector, the lab frame.
The resulting histograms are undistorted by these centre of mass frame boosts
parallel to the beam axis, as the dependent variable is invariant with respect
to this sub–class of Lorentz boosts.

**Paper** (F):

<img src="h1.png" width="500"> <img src="h2.png" width="500">

#### 🔴 **φ Tranformation** <br />
For the φ transformation, we go to each row and subtract the φ of the max pT from all φ's in that row. <br />
If the values exceed [-π, π], we add 2π to the final result (if it's <-π) or subtract 2π from the final result (if it's >π). This makes sure that no values exceed the original φ interval. <br />
This has the effect of making the φ (corresponding to the max pT for that row) equal to 0 in each row, and shifting the other φ's by that same angle, while maintaining a range of 2π. <br />

#### 🔴 Helper Function & Event

In [13]:
def f_transform(event1, max123):
    
    '''
    Input[0]: event (row)
    Input[1]: list of 3 dataframes of max pT, η, φ. Obtained using the extract_max123() function
    '''
    
    # Define φ indices to be used later
    f_indices = mydata_prep.iloc[0][1::3].index
    
    # Create copy of event
    event = event1.copy(deep=True)

    # Transformation (Note: the if statements take periodicity into account, making sure that range does not exceed 2π)
    
    for f_index in f_indices:                    # For all φ in the row
        
        f = event.iloc[1::3][f_index]            # φ original value
        num_index = event.name                   # index of event, so that we can find its corresponding φ in the max123[0] dataframe of max pT's and φ, η's
        k = max123[0].iloc[num_index]['φ']       # φ of max1 pT value
        
    
        if (f - k) < -np.pi:
            event.iloc[1::3][f_index] = f + 2*np.pi - k
    
        elif (f - k) > np.pi:
            event.iloc[1::3][f_index] = f - 2*np.pi - k
   
        else: 
            event.iloc[1::3][f_index] -= k

    
    return event

#### 🔴 Define η', φ' DataFrame

In [11]:
ftrans = []

 
start = time.time()

# Create matrix of transformed events
for i in range(events):
    ftrans.append(f_transform(mydata_prep.iloc[i], max123))

    
end = time.time()

    
# Turn matrix into DataFrame
mydata_prime = pd.DataFrame(ftrans)

print('Time taken: {0:.2f}s'.format(end-start))

Time taken: 82.69s


---
<br />

---
<br />

---
<br />

---
<br />

---

### 🔵(Incomplete) 3. Rotation: Rotate image such that 2nd max pT is at 12 o'clock
Rotate all constituents around (φ’,η’)=0 such that the constituent with the 2nd highest pT is at 12 o’clock, i.e. at  (φ’,η’)=(0,e) with e > 0.

**Paper (C)** <br />
"Rotation: Rotation is performed to remove the stochastic nature of the decay
angle relative to the η − φ coordinate system. This alignment can be done very
generally, by determining the principle axis [48] of the original image and rotating the imagine around the jet-energy centroid such that the principle axis
is always vertical."

#### Resources
https://stackoverflow.com/questions/53854066/pythonhow-to-rotate-an-image-so-that-a-feature-becomes-vertical

https://alyssaq.github.io/2015/computing-the-axes-or-orientation-of-a-blob/

https://pythontic.com/image-processing/pillow/rotate

https://www.askpython.com/python/examples/rotate-an-image-by-an-angle-in-python

https://www.pyimagesearch.com/2017/01/02/rotate-images-correctly-with-opencv-and-python/




---
<br />

---
<br />

---
<br />

---
<br />

---

### 🔵(Incomplete) 4. Flip: Flip image so that 3rd max pT is in right-half plane
Flip all the constituents such that the constituents with the 3rd highest pT is on the right, i.e. at (φ’,η’)=(f,e) with f > 0

---
<br />

---
<br />

---
<br />

---
<br />

---

### 🔵5. Image: Create 40x40 Image
1. Define Data that will create the bins (η, φ) and will weigh the bins (pT)
2. Using the bins, create a 2D histogram using numpy's histogram2d()
3. Input the histogram as a DataFrame into Seaborn's heatmap()
4. Tadaaaa


---

#### 🔴 Define Bins Data (for histogram in the next step)

This is done by combining all rows (events) into a single row for each coordinate (η, φ, pT). <br />

#### 🔴 Old code to check if bins are crated correctly

In [12]:
# Test if bin lengths match
if (len(bin_h) == len(bin_f)) and (len(bin_f) == len(bin_p)):
    print('\nSuccess: Bin lengths match! :D')
else:
    print('\nError: Bin lengths don\'t match!!! :  -  (')
    assert (len(bin_h) == len(bin_f)) and (len(bin_f) == len(bin_p))

# Test if max and min values make sense
print('\nη bin range: [', round(min(bin_h), 2), ', ', round(max(bin_h), 2), ']')
print('φ bin range: [', round(min(bin_f), 2), ', ', round(max(bin_f), 2), ']')
print('p bin range: [', round(min(bin_p), 2), ', ', round(max(bin_p), 2), ']')

NameError: name 'bin_h' is not defined

---

#### 🔴 Create 2D Histogram 🔴 Create Heatmap

---

---